# Zooniverse JSON results to CSV

This Notebook is designed to: #TODO description.

## Assumptions about running this Notebook

We created this Notebook on Apple Macs with Python installed as xxx. If you tweak the Notebook for Python installed via Anaconda or on a different operating system, please feel free to fork and share your changes.

## Making a local copy and installing prerequisites

TODO: How to make a local copy of the notebook, in order to run it on your local computer.

This notebook uses the package `pandas`, which is not built-in to Python. If you don't already have it installed, you will need to run the following command in your command line:

```sh
$ pip install pandas
```

## Downloading Zooniverse data

### 1. Log in

First, [log in to Zooniverse](https://www.zooniverse.org/accounts/sign-in) as you normally would.

### 2. Navigate to lab page

Next, [navigate to the Zooniverse lab page](https://www.zooniverse.org/lab). On the page, you will see the projects where you are a collaborator:

<img src="images/annotations-json-to-csv/1-zooniverse-lab.png?raw=true" alt="A screenshot of the Zooniverse lab page" style="max-width:75%;" title="Screenshot of Zooniverse lab page" />

### 3. Navigate to "Data Exports"

Next, click "Data Exports" in the right-hand menu bar:

<img src="images/annotations-json-to-csv/2-data-exports.png?raw=true" alt="A screenshot that shows where in the right-hand menu bar you will find the Data Exports option" style="max-width:75%;" title="Screenshot of Zooniverse lab splash page" />

### 4. Request relevant exports

Next, you will want to press the two buttons for "Request new classification export" and "Request new subject export":

<img src="images/annotations-json-to-csv/3-data-exports-requests.png?raw=true" alt="A screenshot that shows the relevant buttons to press on the Zooniverse Data Exports page" style="max-width:75%;" title="Screenshot of Zooniverse Data Exports page" />

### 5. Await the completed download

Your export might take a little while, but you will receive an email once your request is completed:

<img src="images/annotations-json-to-csv/4-confirmation-email.png?raw=true" alt="A screenshot that shows the confirmation email received when data export request has been completed" style="max-width:75%;" title="Screenshot of the confirmation email" />

### 6. Download the relevant files

Note that clicking the link "Download from your lab data exports page" in the confirmation email will only take you back to the Data Exports page.

Once you are back to the page, you need to use the correct links on the page to download the relevant CSV files:

<img src="images/annotations-json-to-csv/5-request-completed.png?raw=true" alt="A screenshot that shows the data export request completed on the Data Exports page" style="max-width:75%;" title="Screenshot of the Data Exports page with the relevant links" />

For each of those two files, right-click and choose "Save Link As..."

<img src="images/annotations-json-to-csv/6-save-link-as.png?raw=true" alt="A screenshot that shows the popup menu to save CSV file from your browser in the correct directory" style="max-width:30%;" title="Screenshot of the menu that shows the option Save Link As..." />

Finally, you can save the file wherever you want on your computer, but remember the correct path to the files as you will need to fill them out in the next cell! _We suggest that you save the files with the names `classifications.csv` and `subjects.csv` in your `Downloads` folder._

<img src="images/annotations-json-to-csv/7-rename.png?raw=true" alt="A screenshot that shows the renaming process of the CSV file" style="max-width:30%;" title="Screenshot of the download file dialog" />

Now, in the following cell, which is a Notebook cell registering your variable, we need you to fill in the path to the classifications file (as the `classifications_file` variable) and the subject file (as the `subjects_file` variable).

As an example, if you saved the `classifications.csv` and `subjects.csv` files in the `Downloads` folder above (like in the screenshot), you might want to change the lines below to:

```py
classifications_file = "/Users/<your-username>/Downloads/classifications.csv"
subjects_file = "/Users/<your-username>/Downloads/subjects.csv"
```

In [ ]:
classifications_file = "test-files/test_classifications.csv"
subjects_file = "test-files/test_subjects.csv"

---

## Setting up our definitions

Now, we are ready to get started working with our data.

First, we need to import the packages that we are going to use in the script below. Most of them are built-in to Python.

In [ ]:
from collections import ChainMap, OrderedDict
import hashlib  
import json

import pandas as pd

## Read in our main data

Next, it's time to read in the main CSV data as a pandas DataFrame, using the handy method `.read_csv()`.

In [ ]:
df = pd.read_csv(classifications_file)

Pandas has a handy method to look at the data that we have just imported, called `.head(num)`, where `num` should be replaced with the amount of rows in the frame that you want to see.

We run it below to see what our imported data looks like unprocessed:

In [ ]:
df.head(3)

As we can see in our preview, the `classification_id` is a column, which is unique for each classification. It can thus be used as an "index" for the DataFrame, a nice way of querying the frame by individual IDs.

Here's how we can set the index on our `df`:

In [ ]:
df.set_index("classification_id", inplace=True)

Because Python and Pandas are both "succeeding silently", it can be good to sometimes have what we call a "sanity check" to ensure that we have the result that we want.

In [ ]:
df.head(3)

## Ensure correct encoding

Next, we will want to have a look at the encoding of each column.

In the "sanity check" above, you can see that the columns `subject_data`, `annotations`, and `metadata` are structured as JSON (JavaScript Object Notation), which the DataFrame parser cannot interpret on its own, which is why we have to help it using the following parsing of those specific columns.

If you use Pandas' `.apply()` method (which can be applied on a column or on the entire DataFrame), you can help the software interpret the data in the columns. Here, we pass a function (`json.loads`, i.e. the `loads` function from Python's built-in `json` package) to each of the columns containing JSON data.

In [ ]:
df["metadata"] = df["metadata"].apply(json.loads)
df["annotations"] = df["annotations"].apply(json.loads)

Again, we might want to do a "sanity check". To look at a specific part of a dataframe we can "slice it" by passing it a list (note the double `[[` and `]]` on each side of the selector) of the columns we want to look at:

In [ ]:
df[["annotations", "metadata"]]

Looks good!

In the following two sections, we will process the `metadata`, and `annotations` column respectively.

## Extract classification metadata (`metadata`)

Next, we want to do something similar to what we did with the subjects above to process the metadata for each classification.

Because the data in the `metadata` column has nested JSON data (that is, objects and lists that are wrapped inside each other), we want to use the particular `json_normalize` method.

Here's an example of what one `metadata` row looks like:

In [ ]:
df.metadata[df.metadata.head(1).index[0]]

Now, let's create a separate DataFrame from the normalised JSON data in the columns using the json_normalize method and use `.head()` to see the result:

In [ ]:
df_metadata = pd.json_normalize(df["metadata"])

df_metadata.head(3)

Since we know that the shape of the classification metadata DataFrame (`df_metadata`) and the main classification DataFrame (`df`) are the same, we can apply the `set_index` method to the metadata to get the `classification_id` as index on the `df_metadata`:

In [ ]:
df_metadata.set_index(df.index, inplace=True)

Sanity check time again — good practice, keep track of what you're doing:

In [ ]:
df_metadata.head(3)

## Extract annotations (`annotations`)

The final information we want to extract and process from the main DataFrame are the annotations for each classification.

In order to do so, we must first preprocess the JSON data that we have available, extracting a list of only the information we want (the "annotation values"). We set up a function that processes each individual row, and then apply it to each row, using the `.apply` method, which we have used before:

In [ ]:
def extract_annotation_values(annotation_row):
    """
    Takes an annotation row, which contains a list of tasks with values in dictionary {task, task_label, value}
    and extracts the `value` for each `task`, disregarding the `task_label` and returns them as a dictionary,
    for easy insertion into a DataFrame.
    """
    
    extracted_dictionaries = [{task_data.get('task'): task_data.get('value')} for task_data in annotation_row]
    return dict(ChainMap(*extracted_dictionaries))

df["annotations"] = df["annotations"].apply(extract_annotation_values)

Then, similarly to section 4 above, we loop through each row (of JSON data - contained in `json_data`) and extract the classification_id and annotations for each of them, which we then add on our new DataFrame `df_annotations`:

In [ ]:
# TODO: This cell generates deprecation warning (append is deprecated, use pd.concat instead)

df_annotations = pd.DataFrame()

for index, row in df.iterrows():
    s = pd.Series(row.annotations, name=index)
    df_annotations = df_annotations.append(s)

Finally, we can use the `.head()` method again to check our results:

In [ ]:
df_annotations.head(3)

## Extract Zooniverse subjects (`subject_data`)

Next, we want to read in the `subjects.csv` file above, so we know what files each of the classifications were done on.

In [ ]:
df_subjects = pd.read_csv(subjects_file)

Let's have a quick look at our data using the `.head()` method that we learned above:

In [ ]:
df_subjects.head(3)

As with the classifications, we can see from this preview that the `subject_id` is a column, which is unique for each subject. Similarly to what we did with the classifications above, we can turn it into an "index" for the DataFrame, a nice way of querying the frame by individual IDs. Again, here's how we'd do it:

In [ ]:
df_subjects.set_index("subject_id", inplace=True)

Now, let's see what happened:

In [ ]:
df_subjects.head(3)

As we can see from the preview, there are some quirky aspects of the `subjects.csv` file from Zooniverse. Two things we might want to address here is that the `metadata` column is formatted as embedded JSON data, and so is the `locations` column.

The `metadata` column corresponds to the data from any manifest file that you uploaded with your subject sets, so it often contains valuable information for us to have with our classifications.

The `locations` column contains the URL to all of the images that each subject links to. For our projects, we might just have one location, but not necessarily: subjects can contain multiple images!

For our purposes, we want to create a more readable list of locations and extract the metadata into its own columns. Let's go ahead and do that now.

We will start by making sure that both columns are formatted correctly, as we did with the classifications' metadata and annotations above:

In [ ]:
df_subjects["metadata"] = df_subjects["metadata"].apply(json.loads)
df_subjects["locations"] = df_subjects["locations"].apply(json.loads)

We will start with the `locations` column. Since the data is structured as a Python dictionary `{id: "URL"}`, we can get a list of all the URLs by using Python's `dict` type's built-in `.values()` method. Here, we put that method into a custom function, which takes any row from the `subjects` DataFrame, makes a list from its values, and then joins them together with a comma separation (see the `", ".join()` syntax):

In [ ]:
def locations_as_list(row):
    return ", ".join(list(row.values()))

df_subjects["locations_list"] = df_subjects["locations"].apply(locations_as_list)

Now, to save some space, we can remove the old `locations` columns:

In [ ]:
df_subjects = df_subjects.drop(["locations"], axis="columns")

Next, we will turn to the subjects' `metadata` column.

Similarly to the extraction of classifications metadata above, we will create a separate DataFrame from the normalised JSON data from the subjects column using the `json_normalize` method and then use `.head()` to see the result:

In [ ]:
df_subjects_metadata = pd.json_normalize(df_subjects["metadata"])

df_subjects_metadata.head(3)

Again, since we know that the shape of the subjects' metadata DataFrame (`df_subjects_metadata`) and the main subject DataFrame (`df_subject`) are the same, we can apply the `set_index` method to the metadata to get the `subject_id` as index on the `df_subjects_metadata`.

After, we'll run the `.head()` sanity check, to make sure all is well:

In [ ]:
df_subjects_metadata.set_index(df_subjects.index, inplace=True)

df_subjects_metadata.head(3)

Since we've now extracted the `metadata` column into a separate DataFrame, let's go ahead and remove the column from the original or "main" `df_subjects` DataFrame:

In [ ]:
df_subjects = df_subjects.drop("metadata", axis="columns")

A really great feature of DataFrames is that you can join them back together, as long as the `pandas` library has an easy way of matching the two DataFrames (or more!) together. A joined index, such as we have for `df_subjects` and `df_subjects_metadata` is a great way to do so.

In order to get one large `df_subjects` DataFrame that contains all the data for each subject, we can thus run the `.join` method:

In [ ]:
df_subjects = df_subjects.join(df_subjects_metadata)

Once again, we see that `pandas` succeeds silently, so let's do a sanity check:

In [ ]:
df_subjects.head(3)

That is one large (but handy) DataFrame!

## Joining all the data back together

In this final step, we want to join the data back together into a new main DataFrame, `df_final`, from which we will remove all the personal data so the dataset can be shared publicly, if we want.

To recap, here are the four DataFrames that we have created thus far:

#### Main DataFrame (`df`)

In [ ]:
df.head(1)

#### Subjects DataFrame (`df_subjects`)

In [ ]:
df_subjects.head(1)

#### Metadata DataFrame (`df_metadata`)

In [ ]:
df_metadata.head(1)

#### Annotations DataFrame (`df_annotations`)

In [ ]:
df_annotations.head(1)

### Drop existing data

As you can see from the summary above, of the main DataFrame (`df`), it now doubles the information in `df_subjects`, `df_metadata`, and `df_annotations`. So before we do anything else, we will drop the columns that we have now processed from `df` to reduce the size of `df`:

In [ ]:
df = df.drop(["subject_data", "metadata", "annotations"], axis="columns")

### Redact personal information

Next, we want to ensure that we have anonymised the user data.

Here, we have created our own function, `redact_username`, which uses a cryptographic hash method from Python's built-in `hashlib` library called `sha256`. You can read more about [the algorithm's history on Wikipedia](https://en.wikipedia.org/wiki/SHA-2) if you are interested.

In [ ]:
def redact_username(row):
    return hashlib.sha256(str(row).encode()).hexdigest() if not pd.isna(row) else None
    
df["user_name_redacted"] = df["user_name"].apply(redact_username)

We might also be interested in knowing whether a user was logged in or not when they annotated the subject. In order to preserve that information, we create another custom function, `user_was_logged_in`, which uses the information from the `user_name` column (which has a value like `not-logged-in-98ff168ef257e2fd9d4a` if the user was not logged in) to extract a `True` or `False` value (also called a "boolean"):

In [ ]:
def user_was_logged_in(row):
    return "not-logged-in" not in row if not pd.isna(row) else False

df["user_logged_in"] = df["user_name"].apply(user_was_logged_in)

Finally, we want to drop columns that contain personal identifying information - usernames, IDs and IP addresses - from the main DataFrame:

In [ ]:
df = df.drop(["user_id", "user_name", "user_ip"], axis="columns")

### Join all the data

Now, it's time to join all the DataFrames into one new `df_final` frame.

Adding the metadata and the annotations on the main `df` is an easy thing, since all of them share index:

In [ ]:
df_final = df.join(df_metadata).join(df_annotations)

Let's do a quick sanity check to make sure that all the data is there:

In [ ]:
df_final.head(1)

So far, so good. But now we want to join our subjects on the `df_final` table as well. It's not entirely as easy as the previous multiple join task that we just did here.

This time, because we're not using data points that are uniquely connected, one-to-one (one subject, in fact, will have many classifications), we will end up with duplicated information in the table. In order to join the main DataFrame with the subject DataFrame `df_subjects`, we need to make one intermediary step: We need to make both index columns available in each of the DataFrames.

_Why do we need to make `df_final`'s index a column?_ We will _merge_ two DataFrames, which is a function that will discard both indices of the merging DataFrames.

In [ ]:
df_subjects["subject_ids"] = df_subjects.index
df_final["classification_id"] = df_final.index

Before we run the merge, we also want to make sure that the two matching columns on both DataFrames are of the same (correct) type—otherwise Pandas will have trouble finding the matching subject row for each classification.

For that, we use the built-in method on each DataFrame, `.astype()` which "casts" a column as a type:

In [ ]:
df_subjects["subject_ids"] = df_subjects["subject_ids"].astype(int)
df_final["subject_ids"] = df_final["subject_ids"].astype(int)

Now we should be able to use the `merge` function from Pandas central library to join the two on the column that is shared between the two DataFrames (in our case `subject_ids`):

In [ ]:
df_final = pd.merge(df_final, df_subjects)

Finally, we can make the `classification_id` column an index again, on our final DataFrame:

In [ ]:
df_final = df_final.set_index("classification_id")

One last sanity check, before we're ready to export!

This time, however, we set the max column to be displayed to `None` because we're interested in seeing all the columns. Then we use our now familiar `.head()` method:

In [ ]:
pd.set_option('display.max_columns', None)
df_final.head(3)

## Export!

In this final step, we can export the file to whatever format that we'd like to use.

In the case below, we opt for `.csv`, which is a common, open file format that can be easily opened in Microsoft Excel for further processing. It can also be imported into visualisation software, such as Power BI, or websites like ObservableHQ:

In [ ]:
df_final.to_csv("annotations_unpacked_no_snippets.csv")